## Project title: Public Perception and Engagement Dynamics of the Ukrainian war on Reddit

In [2]:
! pip install -r requirements.txt
! pip install --upgrade pip
! pip install praw

In [3]:
import praw
import requests
import time
from datetime import datetime, timedelta
import pandas as pd

In [4]:
# Reddit API credentials
client_id = "IG6L1mCi0sWNUc1ysvGSEQ"
client_secret = "p-BOlUMb9XPgiJ_mhxpSOnkKBNyvbQ"
user_agent = "MyData"

In [5]:
# Initialize PRAW with credentials
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)

# Access the subreddit
subreddit = reddit.subreddit("UkrainianConflict")

In [6]:
# Retrieve the top 10 hot posts
for post in subreddit.hot(limit=10):
    print(f"Title: {post.title}")
    print(f"Score: {post.score}")
    print(f"Upvote Ratio: {post.upvote_ratio}")
    print(f"Number of Comments: {post.num_comments}")
    print(f"URL: {post.url}\n")

Title: The Daily Hog is back + Discussion Megathread
Score: 69
Upvote Ratio: 0.93
Number of Comments: 14
URL: https://www.reddit.com/r/UkrainianConflict/comments/1dp1abp/the_daily_hog_is_back_discussion_megathread/

Title: Ukrainian Brigade obliterates elite Kadyrov unit in Kursk, shares photographic evidence
Score: 3052
Upvote Ratio: 0.97
Number of Comments: 130
URL: https://www.uawire.org/ukrainian-brigade-obliterates-elite-kadyrov-unit-in-kursk-shares-photographic-evidence

Title: Russian military convoy decimated by HIMARS strike near Kursk: new details emerge on casualties and units involved
Score: 963
Upvote Ratio: 0.99
Number of Comments: 63
URL: https://www.uawire.org/russian-military-column-decimated-by-himars-strike-near-kursk-new-details-emerge-on-casualties-and-units-involved

Title: Robin Brooks "In the first 4 months of 2024, Hungary's exports to Kyrgyzstan were $81 mn versus around $2 mn in the same 4-month period before Russia invaded Ukraine. That's a rise of 3400%. No

In [7]:
# Retrieve the top 10 comments of the first post
top_post = next(subreddit.hot(limit=1))
for comment in top_post.comments:
    if isinstance(comment, praw.models.MoreComments):
        continue
    print(f'Comment by {comment.author}: {comment.body}')
    print(f'Score: {comment.score}\n')

Comment by pkstrl0rd: To any and all Europeans here I wrote this text earlier, and I want you to take action.

I think we need to be much more hawkish than we currently are and think like Russians. We need AT LEAST the readiness to switch to war economy mode fast. No one knows what kind of partner we will have in the US in the future, let alone if a war starts in the Pacific.

It is a great start that Germany will be hosting nukes and some people think, but while I support Germany having those missiles, I hope they are not a crutch they will fall on and prolong true rearmament of Europe, and fast. Besides Germany doesnt even get to decide when one would be launched, the US does. 

So does it get launched when Estonian or Polish first villages or bases fall against an proportionate Russian troop formation or base. Or does it only get launched when defence lines in Poland are about to fall.Or even when Baltic countries have fallen.

Probably not right?

I think ALL NATO countries need to

In [8]:
# Function to get comments from Pushshift.io
def get_old_comments(subreddit, before, after, size=100, limit=1000, max_retries=5):
    url = 'https://api.pushshift.io/reddit/comment/search/'
    all_comments = []
    params = {
        'subreddit': subreddit,
        'before': before,
        'after': after,
        'size': size,
        'sort': 'asc',
        'sort_type': 'created_utc'
    }
    retries = 0

    while len(all_comments) < limit:
        try:
            response = requests.get(url, params=params)
            if response.status_code == 403:
                print('Error 403: Forbidden. Check API status or credentials.')
                break
            elif response.status_code != 200:
                print(f'Error: {response.status_code}. Retrying...')
                retries += 1
                if retries >= max_retries:
                    print('Max retries reached. Exiting...')
                    break
                time.sleep(2 ** retries)  # Exponential backoff
                continue
            data = response.json().get('data', [])
            if not data:
                break
            all_comments.extend(data)
            params['before'] = data[-1]['created_utc']
            print(f"Fetched {len(all_comments)} comments...")
        except requests.exceptions.RequestException as e:
            print(f'Request failed: {e}')
            retries += 1
            if retries >= max_retries:
                print('Max retries reached. Exiting...')
                break
            time.sleep(2 ** retries)  # Exponential backoff

    return all_comments

# Calculate the timestamp for 2 years ago
two_years_ago = datetime.datetime.now() - datetime.timedelta(days=365)
two_years_ago_timestamp = int(two_years_ago.timestamp())

# Calculate the timestamp for 2 years + 1 day ago to limit the search range
two_years_plus_one_day_ago = two_years_ago - datetime.timedelta(days=1)
two_years_plus_one_day_timestamp = int(two_years_plus_one_day_ago.timestamp())

# Get comments from 2 years ago in the 'UkrainianConflict' subreddit
subreddit = 'UkrainianConflict'
comments = get_old_comments(subreddit, before=two_years_ago_timestamp, after=two_years_plus_one_day_timestamp)

# Check if comments were found and print the earliest comment's date
if comments:
    earliest_comment_time = datetime.datetime.fromtimestamp(comments[0]['created_utc'])
    earliest_comment_date = earliest_comment_time.strftime('%d.%m.%Y')
    print(f'Earliest comment date: {earliest_comment_date}')
else:
    print('No comments found within the specified time range.')

# Print details of retrieved comments
for comment in comments:
    print(f'Comment ID: {comment["id"]}')
    print(f'Author: {comment["author"]}')
    print(f'Body: {comment["body"]}')
    print(f'Created at: {datetime.datetime.fromtimestamp(comment["created_utc"]).strftime("%d.%m.%Y %H:%M:%S")}\n')

    

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [ ]:
# Specify the subreddit
subreddit = reddit.subreddit('UkrainianConflict')

# Calculate the time window (up to five days ago)
one_year_ago = datetime.utcnow() - timedelta(days=365)

# Retrieve submissions and comments
for submission in subreddit.new(limit=None):
    # Check if the submission is within the time window
    if submission.created_utc >= one_year_ago.timestamp():
        print(f"Submission Title: {submission.title}")
        print(f"Submission URL: {submission.url}")
        print(f"Submission Author: {submission.author}")
        print(f"Submission Created Time (UTC): {datetime.utcfromtimestamp(submission.created_utc)}")
        print(f"Number of Comments: {submission.num_comments}")
        print("\nComments:")
        
        # Fetch comments for the submission
        submission.comments.replace_more(limit=None)
        for comment in submission.comments.list():
            # Print only the top-level comments
            if comment.depth == 0:
                print(f"Comment by {comment.author}: {comment.body}")
        
        print("\n------------------------\n")

In [23]:
# Define the subreddit and time window
subreddit_name = 'UkrainianConflict'
subreddit = reddit.subreddit(subreddit_name)
five_days_ago = datetime.utcnow() - timedelta(days=5)

# Lists to store the data
posts_data = []

# Fetch posts from the last 5 days
for submission in subreddit.new(limit=None):
    if datetime.utcfromtimestamp(submission.created_utc) >= five_days_ago:
        post = {
            'id': submission.id,
            'title': submission.title,
            'score': submission.score,
            'num_comments': submission.num_comments,
            'upvote_ratio': submission.upvote_ratio,
            'created_utc': datetime.utcfromtimestamp(submission.created_utc),
            'author': submission.author.name if submission.author else 'deleted',
            'url': submission.url
        }
        posts_data.append(post)

# Create a DataFrame
df = pd.DataFrame(posts_data)

# Save the DataFrame to a CSV file
csv_file = 'reddit_ukrainian_conflict_last_5_days.csv'
df.to_csv(csv_file, index=False)

print(f"Data saved to {csv_file}")

Data saved to reddit_ukrainian_conflict_last_5_days.csv


## Subreddit r/Ukrainian_Conflict

In [24]:
# Define the subreddit and time window
subreddit_name = 'UkrainianConflict'
subreddit = reddit.subreddit(subreddit_name)
five_days_ago = datetime.utcnow() - timedelta(days=30)

# Lists to store the data
posts_data = []

# Fetch posts from the last 5 days
for submission in subreddit.new(limit=None):
    if datetime.utcfromtimestamp(submission.created_utc) >= five_days_ago:
        post = {
            'id': submission.id,
            'title': submission.title,
            'score': submission.score,
            'num_comments': submission.num_comments,
            'upvote_ratio': submission.upvote_ratio,
            'created_utc': datetime.utcfromtimestamp(submission.created_utc),
            'author': submission.author.name if submission.author else 'deleted',
            'url': submission.url
        }
        posts_data.append(post)

# Create a DataFrame
df = pd.DataFrame(posts_data)

# Save the DataFrame to a CSV file
csv_file = 'Data gathering/reddit_ukrainian_conflict_last_30_days___5.csv'
df.to_csv(csv_file, index=False)

print(f"Data saved to {csv_file}")

Data saved to Data gathering/reddit_ukrainian_conflict_last_30_days___5.csv


## Subreddit r/ukraine last 2 weeks

In [25]:
#NEW SUBREDDIT
# Define the subreddit and time window
subreddit_name = ('ukraine')
subreddit = reddit.subreddit(subreddit_name)
five_days_ago = datetime.utcnow() - timedelta(days=30)

# Lists to store the data
posts_data = []

# Fetch posts from the last 5 days
for submission in subreddit.new(limit=None):
    if datetime.utcfromtimestamp(submission.created_utc) >= five_days_ago:
        post = {
            'id': submission.id,
            'title': submission.title,
            'score': submission.score,
            'num_comments': submission.num_comments,
            'upvote_ratio': submission.upvote_ratio,
            'created_utc': datetime.utcfromtimestamp(submission.created_utc),
            'author': submission.author.name if submission.author else 'deleted',
            'url': submission.url
        }
        posts_data.append(post)

# Create a DataFrame
df = pd.DataFrame(posts_data)

# Save the DataFrame to a CSV file
csv_file = 'Data gathering/reddit_ukraine_last_30_days___4.csv'
df.to_csv(csv_file, index=False)

print(f"Data saved to {csv_file}")

Data saved to Data gathering/reddit_ukraine_last_30_days___4.csv


## Subreddit r/ukraine June first 2 weeks

In [ ]:
def fetch_reddit_data(subreddit, start_epoch, end_epoch, size=100):
    base_url = 'http://api.pushshift.io/reddit/search/submission'
    all_posts = []
    headers = {'User-Agent': 'MyRedditScraper/1.0'}
    while start_epoch < end_epoch:
        params = {
            'subreddit': subreddit,
            'after': start_epoch,
            'before': end_epoch,
            'size': size,
            'sort': 'asc',  # sort by creation time ascending
            'sort_type': 'created_utc'
        }
        response = requests.get(base_url, headers=headers, params=params)
        if response.status_code == 200:
            posts = response.json()['data']
            if not posts:
                break
            all_posts.extend(posts)
            start_epoch = posts[-1]['created_utc'] + 1  # Avoid duplicate entries
        else:
            print(f"Error fetching data: {response.status_code}")
            time.sleep(60)  # Wait a minute before retrying
            continue
        time.sleep(1)  # Avoid hitting rate limits
    return all_posts

# Define the time range for June 1, 2024, to June 14, 2024
subreddit = 'ukraine'
start_epoch = int(time.mktime(time.strptime('2024-06-01', '%Y-%m-%d')))
end_epoch = int(time.mktime(time.strptime('2024-06-14', '%Y-%m-%d')))

# Fetch the data
posts = fetch_reddit_data(subreddit, start_epoch, end_epoch)
df = pd.DataFrame(posts)

# Save to CSV
df.to_csv('reddit_ukraine_june_2024.csv', index=False)

Error fetching data: 403
Error fetching data: 403
Error fetching data: 403
Error fetching data: 403
Error fetching data: 403
Error fetching data: 403
Error fetching data: 403
Error fetching data: 403
Error fetching data: 403
Error fetching data: 403
Error fetching data: 403


## Subreddit r/geopolitics 30 days

In [ ]:
#NEW SUBREDDIT
# Define the subreddit and time window
subreddit_name = ('geopolitics')
subreddit = reddit.subreddit(subreddit_name)
thirty_days_ago = datetime.utcnow() - timedelta(days=30)

# Lists to store the data
posts_data = []

# Fetch posts from the last 5 days
for submission in subreddit.new(limit=None):
    if datetime.utcfromtimestamp(submission.created_utc) >= thirty_days_ago:
        post = {
            'id': submission.id,
            'title': submission.title,
            'score': submission.score,
            'num_comments': submission.num_comments,
            'upvote_ratio': submission.upvote_ratio,
            'created_utc': datetime.utcfromtimestamp(submission.created_utc),
            'author': submission.author.name if submission.author else 'deleted',
            'url': submission.url
        }
        posts_data.append(post)

# Create a DataFrame
df = pd.DataFrame(posts_data)

# Save the DataFrame to a CSV file
csv_file = 'reddit_geopolitics_last_30_days.csv'
df.to_csv(csv_file, index=False)

print(f"Data saved to {csv_file}")

## Subreddit r/geopolitics 3 months

In [10]:
#NEW SUBREDDIT
# Define the subreddit and time window
subreddit_name = ('geopolitics')
subreddit = reddit.subreddit(subreddit_name)
ninety_days_ago = datetime.utcnow() - timedelta(days=90)

# Lists to store the data
posts_data = []

# Fetch posts from the last 5 days
for submission in subreddit.new(limit=None):
    if datetime.utcfromtimestamp(submission.created_utc) >= ninety_days_ago:
        post = {
            'id': submission.id,
            'title': submission.title,
            'score': submission.score,
            'num_comments': submission.num_comments,
            'upvote_ratio': submission.upvote_ratio,
            'created_utc': datetime.utcfromtimestamp(submission.created_utc),
            'author': submission.author.name if submission.author else 'deleted',
            'url': submission.url
        }
        posts_data.append(post)

# Create a DataFrame
df = pd.DataFrame(posts_data)

# Save the DataFrame to a CSV file
csv_file = 'Data gathering/reddit_geopolitics.csv'
df.to_csv(csv_file, index=False)

print(f"Data saved to {csv_file}")

Data saved to Data gathering/reddit_geopolitics.csv
